In [73]:
#!cp 0-sequential.py 1-input.py
!chmod +x *.py

In [79]:
#!/usr/bin/env python3

#build_model = __import__('0-sequential').build_model
import tensorflow as tf
import tensorflow.keras as K


def build_modell(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    for i in range(len(layers)):
        init = (tf.contrib.layers.
            variance_scaling_initializer(mode="FAN_AVG"))
        freg = K.layers.ActivityRegularization(l2=lambtha)
        layer = K.layers.Dense(layers[i], input_dim=nx,
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=keep_prob)
            model.add(dropped)
    return (model)

def build_model(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    for i in range(len(layers)):
        init = K.initializers.VarianceScaling(mode="fan_avg")
        freg = K.layers.ActivityRegularization(l2=lambtha)
        layer = K.layers.Dense(layers[i],
                               activation=activations[i],
                               kernel_initializer=init,
                               kernel_regularizer=freg)
        model.add(layer)
        if i != len(layers)-1:
            dropped = K.layers.Dropout(rate=keep_prob)
            model.add(dropped)
    model.build((None, nx))
    return (model)

if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

ValueError: This model has never been called, thus its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some data).